# Diffusion and Oxidant Concentration

This plots the change in several different aerosol properties as a function of diffusion coefficient and oxidant concentration. The diffusion coefficient is increased by an order of magnitude for each model scenario. The number after D indicates the negative base 10 logarithm of the diffusion coefficient for that scenario. The oxidant concentration is indicated by the condition where ft corresponds to flow tube ($5.04 \times 10^{10}$ molecules cm$^{-3}$), cfstr corresponds to continuous flow stir reactor ($5.04 \times 10^{8}$ molecules cm$^{-3}$), and day corresponds to daytime conditions ($5 \times 10^{6}$ molecules cm$^{-3}$). The data is first loaded in and processed, and then panel plots of each condition are produced. See Postprocesing.ipynb for a more detailed explanation of the data processing.

%%HTML 
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [14]:
# Import modules

import warnings
warnings.filterwarnings('ignore')
import os
import zipfile

import StackSim
import numpy as np
import pandas as pd
from matplotlib import ticker
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import re



In [10]:
# Create subdirectory for data if it does not exist

subdir = os.path.join(os.path.curdir, "difdata")

if not os.path.isdir(subdir):
    os.mkdir(subdir)
 

# Unzip XML files (.zip file available on Github repository)

with zipfile.ZipFile("diffusion series.zip", "r") as zip_ref:
    zip_ref.extractall(subdir)


In [17]:

diffusion = ["D15", "D16", "D17", "D18"]
condition = ["ft", "cfstr", "day"]

# Generate filename strings

files = dict()

for cond in condition:
    for coefficient in diffusion:
        filename = "tri_"+cond+"_"+coefficient+".xml"
        files[cond+"_"+coefficient] = filename

# Additional conditions for flow tube and CFSTR
files["ft_D12"] = "tri_ft_D12.xml"
files["ft_D13"] = "tri_ft_D13.xml"
files["ft_D14"] = "tri_ft_D14.xml"
files["cfstr_D13"] = "tri_cfstr_D13.xml"
files["cfstr_D14"] = "tri_cfstr_D14.xml"

# create dictionary for reaction conditions
conditions = {"ft" : (5.04E10, 60, "seconds"),
             "cfstr" : (5.04E8, 10, "hours"),
             "day" : (5E6, 7, "days")}

In [18]:
# Generate lists for finding weighting aggregate species from individual
# functional group or carbon backbone species (See Postprocessing.ipynb for more details)

carbon_min, carbon_max = 1, 30

missing_carbon_no = [1, 30]

# Create carbon-30 lumped species from each component

carbon_no_dict = {"nC30" : [["C30", 1], ["C30_COOH", 1], ["C30_COOH_O", 1], ["C30_HOOCCOOH", 1], ["C30_O2", 1], ["C30_O", 1]]}

# Initialize list of lists

carbon_list = [["nC30", 30]]

for i in range(carbon_min, carbon_max+1, 1):
    if i in missing_carbon_no:
        pass
    else:
        no_C = "C"+str(i)
        
        # Add entry to lumped carbon number dictionary
        carbon_no_dict["n"+no_C] = [[no_C, 1], [no_C+"_O2", 1], [no_C+"_COOH", 1], [no_C+"_COOH_O", 1], [no_C+"_HOOCCOOH", 1]]
        
        # Add entry to list of list for carbon number weighting
        carbon_list.append(["n"+no_C, i])

# Generate oxygen and hydrogen species weighting lists

oxygen_list = [["OC_sec", 1], ["OCH_prim", 1], ["OHCH2_prim", 1], ["OHCH_sec", 1],
               ["OC_alpha", 1], ["OHCH_alpha", 1], ["HO_OOC_prim", 3],
               ["HOOCH2_prim", 2], ["HOOCH_alpha", 2], ["HOOCH_sec", 2], ["HOOC_prim", 2]]

hydrogen_list = [["CH3_prim", 3], ["CH3_prim_s", 3], ["CH2_sec", 2], ["CH2_alpha", 2], 
                 ["OCH_prim", 1], ["OHCH_sec", 2], ["OHCH2_prim", 3], ["OHCH_alpha", 2],
                 ["HO_OOC_prim", 1], ["HOOCH2_prim", 3], ["HOOCH_alpha", 2], ["HOOCH_sec", 2], ["HOOC_prim", 1]]

# Generate mass weighting lists

mass_list = [["carbon", 12], ["oxygen", 16], ["hydrogen", 1]]

mass_list_r = [["carbon_r", 12], ["oxygen_r", 16], ["hydrogen_r", 1]]



In [21]:
# Load each condition and diffusion coefficient combination

Scenarios = dict()

for scenario in files.keys():
    
    # Load and process simulation data from simulation    
    Scenarios[scenario] = StackSim.SimulationData(os.path.join(subdir, files[scenario]))
    
    # Calculate aggregate carbon species and radial correction
    for carbon_no in carbon_no_dict.keys():
        Scenarios[scenario].calcAggregateSpecies(carbon_no, carbon_no_dict[carbon_no])
        Scenarios[scenario].calcRadialCorrection(carbon_no, reverse_axis=True)
    

    # Calculate each element and mass    
    Scenarios[scenario].calcAggregateSpecies("carbon", carbon_list)

    Scenarios[scenario].calcAggregateSpecies("oxygen", oxygen_list)

    Scenarios[scenario].calcAggregateSpecies("hydrogen", hydrogen_list)     
    
    Scenarios[scenario].calcAggregateSpecies("mass", mass_list)
    
    # Calculate radial correction for each element and mass    
    
    Scenarios[scenario].calcRadialCorrection("mass", reverse_axis=True)
    Scenarios[scenario].calcRadialCorrection("oxygen", reverse_axis=True)
    Scenarios[scenario].calcRadialCorrection("carbon", reverse_axis=True)
    Scenarios[scenario].calcRadialCorrection("hydrogen", reverse_axis=True)
    
    # Calculate radial correction for triacontane    
    
    Scenarios[scenario].calcRadialCorrection("Tri", reverse_axis=True)
    
    # Calculate mass of aerosol    
    
    Scenarios[scenario].calcAggregateSpecies("mass_r", mass_list_r)
    
    Scenarios[scenario].calcSpeciesRatio("O/C ratio", "oxygen_r", "carbon_r")
    Scenarios[scenario].calcSpeciesRatio("H/C ratio", "hydrogen_r", "carbon_r")

## Panel plots

After processing the data, panel plots showing several different variables at different oxidant and diffusion coefficients are generated.